In [ ]:
import pickle
import numpy as np
from collections import defaultdict

from Venue import *
from Content import *

## Create Venues

In [ ]:
venue_id = pickle.load(open('venue/venue_id.pkl', 'rb'))
venue_pool = pickle.load(open('venue/venue_pool.pkl', 'rb'))

id_venue, removes = {}, set()
for venue, vid in venue_id.items():
    if vid in venue_pool:
        id_venue[vid] = venue
    else:
        removes.add(venue)
        
for venue in removes:
    del venue_id[venue]
    
pickle.dump(venue_id, open('venue/venue_id.pkl', 'wb'), -1)
pickle.dump(id_venue, open('venue/id_venue.pkl', 'wb'), -1)
print(len(venue_pool), len(id_venue), len(venue_id))

In [ ]:
start_year, end_year = 1980, 2019
content_authors = pickle.load(open('content/content_authors.pkl','rb'))

venues = {}
for year in range(start_year, end_year):
    contents = pickle.load(open('content/contents_{}.pkl'.format(year),'rb'))        
    for content in contents.values():
        
        if content.venue not in venues:
            data = {'id':content.venue, 'name':id_venue[content.venue]}
            venues[content.venue] = Venue(data)
            
        venues[content.venue].update_contents(year, content.id, content.field, content_authors[content.id])
        
    print(f'Venue done for year {year}')
    
for venue in venues.values():
    venue.update_authors()
    venue.update_fields()

pickle.dump(venues, open('venue/venues.pkl', 'wb'), -1)
print(len(venues))

In [ ]:
venues = pickle.load(open('venue/venues.pkl','rb'))
venue_cumcounts, venue_fields = defaultdict(np.int64), defaultdict(dict)

for venue in venues.values():
    for year, field in venue.fields.items():
        venue_fields[venue.id][year] = field/np.linalg.norm(field,2)
    first_year = max(1999, min(venue.contents.keys()))
    for year in range(first_year,2019):
        venue_cumcounts[year] += 1
        
pickle.dump(venue_cumcounts, open('venue/venue_cumcounts.pkl', 'wb'), -1)
pickle.dump(venue_fields, open('venue/venue_fields.pkl', 'wb'), -1)

## Embeddings

In [ ]:
u_embs = pickle.load(open('venue/v_embs.pkl','rb'))
venues = pickle.load(open('venue/venues.pkl','rb'))

update_counts = 0
for venue in venues.values():        
    for year in venue.contents.keys():
        venue.update_embeddings(year, u_embs[venue.id+str(year)])
        update_counts += 1
    
pickle.dump(venues, open('venue/venues.pkl', 'wb'), -1)
print('Count check: {} vs {}'.format(len(u_embs), update_counts))